### Import necessary spark functions.

In [110]:
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }

        kernel.publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)        
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<console>:41: error: not found: value kernel
        kernel.publish.html(s""" <table>
        ^



In [59]:
import org.apache.spark.sql.functions._ 
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.sql._
import org.apache.spark.sql.expressions.Window

org.apache.spark.sql.expressions.Window$@5de0f7ad

https://kokes.github.io/blog/2018/05/19/spark-sane-csv-processing.html

In [4]:
import org.apache.spark.sql.types._

val customSchema = StructType(Array(
    StructField("PostId", DoubleType, true),
    StructField("PostCreationDate", StringType, true),
    StructField("OwnerUserId", DoubleType, true),
    StructField("OwnerCreationDate", StringType, true),
    StructField("ReputationAtPostCreation", DoubleType, true),
    StructField("OwnerUndeletedAnswerCountAtPostTime", DoubleType, true),
    StructField("Title", StringType, true),
    StructField("BodyMarkdown", StringType, true),
    StructField("Tag1", StringType, true),
    StructField("Tag2", StringType, true),
    StructField("Tag3", StringType, true),
    StructField("Tag4", StringType, true),
    StructField("Tag5", StringType, true),
    StructField("PostClosedDate", StringType, true),
    StructField("OpenStatus", StringType, true)
))

var df = spark.
    read.
    option("quote", "\"").
    option("escape", "\"").
    option("multiLine", "true").
    option("header", "true").
    schema(customSchema).
    csv("train-sample.csv")

customSchema = StructType(StructField(PostId,DoubleType,true), StructField(PostCreationDate,StringType,true), StructField(OwnerUserId,DoubleType,true), StructField(OwnerCreationDate,StringType,true), StructField(ReputationAtPostCreation,DoubleType,true), StructField(OwnerUndeletedAnswerCountAtPostTime,DoubleType,true), StructField(Title,StringType,true), StructField(BodyMarkdown,StringType,true), StructField(Tag1,StringType,true), StructField(Tag2,StringType,true), StructField(Tag3,StringType,true), StructField(Tag4,StringType,true), StructField(Tag5,StringType,true), StructField(PostClosedDate,StringType,true), StructField(OpenStatus,StringType,true))
df = [PostId: double, PostCre...


[PostId: double, PostCre...

### Read sample of training data set in from S3 bucket.

In [81]:
import org.apache.spark.sql.types._

val customSchema = StructType(Array(
    StructField("PostId", DoubleType, true),
    StructField("PostCreationDate", StringType, true),
    StructField("OwnerUserId", DoubleType, true),
    StructField("OwnerCreationDate", StringType, true),
    StructField("ReputationAtPostCreation", DoubleType, true),
    StructField("OwnerUndeletedAnswerCountAtPostTime", DoubleType, true),
    StructField("Title", StringType, true),
    StructField("BodyMarkdown", StringType, true),
    StructField("Tag1", StringType, true),
    StructField("Tag2", StringType, true),
    StructField("Tag3", StringType, true),
    StructField("Tag4", StringType, true),
    StructField("Tag5", StringType, true),
    StructField("PostClosedDate", StringType, true),
    StructField("OpenStatus", StringType, true)
))

var df = spark.
    read.
    option("quote", "\"").
    option("escape", "\"").
    option("multiLine", "true").
    option("header", "true").
    schema(customSchema).
    csv("s3://stackoverflow-kaggle/data/train-sample.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.types._
customSchema: org.apache.spark.sql.types.StructType = StructType(StructField(PostId,DoubleType,true), StructField(PostCreationDate,StringType,true), StructField(OwnerUserId,DoubleType,true), StructField(OwnerCreationDate,StringType,true), StructField(ReputationAtPostCreation,DoubleType,true), StructField(OwnerUndeletedAnswerCountAtPostTime,DoubleType,true), StructField(Title,StringType,true), StructField(BodyMarkdown,StringType,true), StructField(Tag1,StringType,true), StructField(Tag2,StringType,true), StructField(Tag3,StringType,true), StructField(Tag4,StringType,true), StructField(Tag5,StringType,true), StructField(PostClosedDate,StringType,true), StructField(OpenStatus,StringType,true))
df: org.apache.spark.sql.DataFrame = [PostId: double, PostCreationDate: string ... 13 more fields]


In [5]:
df.show()

+-----------+-------------------+-----------+-------------------+------------------------+-----------------------------------+--------------------+--------------------+---------------+-----------------+-------------+------------------+----------+-------------------+--------------------+
|     PostId|   PostCreationDate|OwnerUserId|  OwnerCreationDate|ReputationAtPostCreation|OwnerUndeletedAnswerCountAtPostTime|               Title|        BodyMarkdown|           Tag1|             Tag2|         Tag3|              Tag4|      Tag5|     PostClosedDate|          OpenStatus|
+-----------+-------------------+-----------+-------------------+------------------------+-----------------------------------+--------------------+--------------------+---------------+-----------------+-------------+------------------+----------+-------------------+--------------------+
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
+-----------+-------------------+-----------+-------------------+------------------------+------

Get the percentage of null values for each column -- Tag2, Tag3, and Tag4 are the only columns with a significant amount of missing data.

In [48]:
df = df.withColumn("OpenStatus", regexp_extract(col("OpenStatus"), "([\\w\\s]+\\w)", 1))

df = [PostId: double, PostCreationDate: string ... 13 more fields]


[PostId: double, PostCreationDate: string ... 13 more fields]

In [6]:
df.
    select(df.columns.map(c => (sum(when(col(c).isNull || col(c) === "" || col(c).isNaN, 1).otherwise(0)) / df.count()).alias(c)): _*).
    show

+------+----------------+-----------+-----------------+------------------------+-----------------------------------+-----+------------+--------------------+-------------------+-------------------+------------------+------------------+--------------+----------+
|PostId|PostCreationDate|OwnerUserId|OwnerCreationDate|ReputationAtPostCreation|OwnerUndeletedAnswerCountAtPostTime|Title|BodyMarkdown|                Tag1|               Tag2|               Tag3|              Tag4|              Tag5|PostClosedDate|OpenStatus|
+------+----------------+-----------+-----------------+------------------------+-----------------------------------+-----+------------+--------------------+-------------------+-------------------+------------------+------------------+--------------+----------+
|   0.0|             0.0|        0.0|              0.0|                     0.0|                                0.0|  0.0|         0.0|7.129006501653929E-6|0.19408720200752824|0.45855195619938405|0.7172208281053952|0.

In [7]:
df = df.na.fill("unknown", Seq("Tag1", "Tag2", "Tag3", "Tag4", "Tag5"))

df = [PostId: double, PostCreationDate: string ... 13 more fields]


[PostId: double, PostCreationDate: string ... 13 more fields]

## Simple model: Random Forest with Tags as features

To start with, we'll take a simple approach - encode the Tag columns as categorical variables and predict `OpenStatus` using Random Forest. Spark's RF Classifier example will serve as a reference ([link](https://github.com/apache/spark/blob/master/examples/src/main/scala/org/apache/spark/examples/ml/RandomForestClassifierExample.scala)).

In [50]:
val df_simple = df.
    select(col("Tag1"), col("Tag2"), col("Tag3"), col("Tag4"), col("Tag5"), col("OpenStatus"))

df_simple = [Tag1: string, Tag2: string ... 4 more fields]


[Tag1: string, Tag2: string ... 4 more fields]

### EDA - frequency of `OpenStatus` and `Tag1`

We see that `null` is the most often observed response by a large margin --- our data set is very imbalanced.

In [51]:
val status_counts = df_simple.
    groupBy("OpenStatus").
    count().
    sort(col("count").desc)

status_counts.show(numRows = 10)

+-------------------+-----+
|         OpenStatus|count|
+-------------------+-----+
|               open|70136|
|not a real question|30789|
|          off topic|17530|
|   not constructive|15659|
|      too localized| 6158|
+-------------------+-----+



status_counts = [OpenStatus: string, count: bigint]


[OpenStatus: string, count: bigint]

As it turns out pretty much every null `OpenStatus` is paired with a null `Tag1` (at leas in our sample training set).

Note: in the join the use of `<=>` allows for joining on null values of `tag1` ([so_link](https://stackoverflow.com/a/41729359)).

In [52]:
val tag1_counts = df_simple.
    groupBy("Tag1").
    count().
    withColumnRenamed("count", "tag1_count").
    orderBy(col("count").desc)

val tag1_status = df_simple.
    groupBy("Tag1", "OpenStatus").
    count().
    join(tag1_counts, Seq("Tag1"), joinType = "left").
    where(col("count") > 20).
    orderBy(col("tag1_count").desc_nulls_first, col("count").desc_nulls_first)

tag1_status.show()

+-------+-------------------+-----+----------+
|   Tag1|         OpenStatus|count|tag1_count|
+-------+-------------------+-----+----------+
|    php|               open| 4781|     12499|
|    php|not a real question| 4185|     12499|
|    php|   not constructive| 1485|     12499|
|    php|          off topic| 1032|     12499|
|    php|      too localized| 1016|     12499|
|     c#|               open| 5831|     11322|
|     c#|not a real question| 3372|     11322|
|     c#|   not constructive| 1194|     11322|
|     c#|      too localized|  502|     11322|
|     c#|          off topic|  423|     11322|
|   java|               open| 4923|     11146|
|   java|not a real question| 3499|     11146|
|   java|   not constructive| 1562|     11146|
|   java|          off topic|  588|     11146|
|   java|      too localized|  574|     11146|
|android|               open| 3735|      7175|
|android|not a real question| 1974|      7175|
|android|          off topic|  761|      7175|
|android|   n

tag1_counts = [Tag1: string, tag1_count: bigint]
tag1_status = [Tag1: string, OpenStatus: string ... 2 more fields]


[Tag1: string, OpenStatus: string ... 2 more fields]

In [68]:
val cumsum_window = Window.
  orderBy(col("count").desc).
  rowsBetween(Window.unboundedPreceding, Window.currentRow)

val total_window = Window.
    rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

df_simple.
    groupBy("Tag1").
    count().
    orderBy(col("count").desc).
    withColumn("fracObs", sum(col("count")).over(cumsum_window) / sum(col("count")).over(total_window)).
    filter(col("fracObs") <= 0.90).
    count()

cumsum_window = org.apache.spark.sql.expressions.WindowSpec@7c6f4283
total_window = org.apache.spark.sql.expressions.WindowSpec@4e033066


382

### Pipeline for Random Forest

Since there are null `Tag1` values, we use `setHandleInvalid("keep")` so that they are indexed rather than dropped ([so_link](https://stackoverflow.com/a/36113473)).

In [40]:
val tag1Indexer = new StringIndexer().
    setInputCol("Tag1").
    setOutputCol("indexedTag1").
    setHandleInvalid("keep").
    fit(simple_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tag1Indexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_ba0487a209f3


In [42]:
tag1Indexer.
    transform(simple_train).
    select("Tag1", "indexedTag1").
    show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+
|Tag1|indexedTag1|
+----+-----------+
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
|null|    10609.0|
+----+-----------+
only showing top 20 rows



We want to create a StringIndexer for each Tag column; rather than create 5 variables we'll take a functional approach:

In [70]:
val featureCols = Array[String]("Tag1", "Tag2", "Tag3", "Tag4", "Tag5")

val featureIndexers = featureCols.map { colName =>
    new StringIndexer().
        setInputCol(colName).
        setOutputCol("indexed" + colName).
        setHandleInvalid("keep").
        fit(simple_train)
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

featureCols: Array[String] = Array(Tag1, Tag2, Tag3, Tag4, Tag5)
featureIndexers: Array[org.apache.spark.ml.feature.StringIndexerModel] = Array(strIdx_3f2aedc88dcd, strIdx_b80161de7b98, strIdx_285c550b6cee, strIdx_a152531a9eae, strIdx_db6635162996)


Spark ML models expect a feature vector to be the only predictor.  [`VectorAssembler`](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler) is a transformer that combines a list of columns into a single vector column.

In [54]:
val assembler = new VectorAssembler().
    setInputCols(featureCols.map{x => "indexed" + x}).
    setOutputCol("features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_47f54c17f830


Just as with the categorical features, we index the response:

In [71]:
val labelIndexer = new StringIndexer().
    setInputCol("OpenStatus").
    setOutputCol("label").
    setHandleInvalid("keep").
    fit(simple_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_763c47d16a69


Since the response is indexed, we need a way to transform the predicted response back to its original string value.  This inverse transformer is called [`IndexToString`](https://spark.apache.org/docs/latest/ml-features.html#indextostring):

In [72]:
val labelConverter = new IndexToString().
    setInputCol("prediction").
    setOutputCol("predictionLabel").
    setLabels(labelIndexer.labels)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.IndexToString = idxToStr_16829d1c7c43


Finally we can specify our model, a [`RandomForestClassifer`](https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier):

In [75]:
val rf = new RandomForestClassifier().
    setLabelCol("label").
    setFeaturesCol("features").
    setNumTrees(10).
    setMaxBins(12000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_9df6a58d40ea


In [76]:
val pipeline = new Pipeline().
    setStages(featureIndexers ++ Array(assembler, labelIndexer, rf, labelConverter))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pipeline: org.apache.spark.ml.Pipeline = pipeline_b75699e47b54


### Fit Pipeline

In [68]:
val Array(trainingData, testData) = simple_train.randomSplit(Array(0.7, 0.3))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Tag1: string, Tag2: string ... 4 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Tag1: string, Tag2: string ... 4 more fields]


In [77]:
val model = pipeline.fit(trainingData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model: org.apache.spark.ml.PipelineModel = pipeline_b75699e47b54


In [79]:
val predictions = model.transform(testData)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [Tag1: string, Tag2: string ... 15 more fields]


In [81]:
predictions.select("predictionLabel", "label", "features").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 616.0 failed 4 times, most recent failure: Lost task 0.3 in stage 616.0 (TID 6026, ip-172-31-8-106.us-west-2.compute.internal, executor 26): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$10: (double) => string)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:291)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:283)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$24.apply(RDD.scala:836)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun

In [78]:
val evaluator = new MulticlassClassificationEvaluator()
    .setLabelCol("indexedLabel")
    .setPredictionCol("prediction")
    .setMetricName("accuracy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_a06798eca4b7
